# Shakespearean Tweet Generation Agents
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MervinPraison/PraisonAI/blob/main/cookbooks/notebooks/shakespearean_tweets_generation_agents.ipynb)

## Dependencies

In [1]:
# Install dependencies without output
%pip install langchain_community > /dev/null
%pip install praisonai[crewai] > /dev/null


## Tools

In [26]:
from praisonai_tools import BaseTool
import re

class CharacterCounterTool(BaseTool):
    name: str = "CharacterCounterTool"
    description: str = "Counts characters in a text and checks if it meets specific character limits."

    def _run(self, text: str, min_length: int = 200, max_length: int = 280) -> dict:
        """
        Analyzes the provided text to check if it meets the specified character limits and
        whether it contains any forbidden emojis.

        Parameters:
        - text (str): The text to analyze.
        - min_length (int): The minimum allowed character count. Default is 200.
        - max_length (int): The maximum allowed character count. Default is 280.

        Returns:
        - dict: A dictionary with validation results, including character count, emoji presence,
                and compliance with the specified character range.
        """
        char_count = len(text)
        within_limits = min_length <= char_count <= max_length
        emoji_pattern = re.compile(
            r"[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F700-\U0001F77F]"
        )
        contains_emoji = bool(emoji_pattern.search(text))

        return {
            "character_count": char_count,
            "within_limits": within_limits,
            "contains_emoji": contains_emoji,
        }


## YAML Prompt

In [30]:
agent_yaml = """
framework: "crewai"
topic: "Shakespearean-Style X Post Generation and Verification"
roles:
  shakespearean_bard:
    role: "Shakespearean Bard"
    backstory: |
      A witty bard, renowned for turning the mundane into the magnificent with playful jests and biting sarcasm.
      Armed with wit and wisdom, you revel in creating humorous quips most pleasing to the ear.
    goal: |
      Craft sarcastic and playful hot takes in the style of Shakespeare that fit within modern social media constraints.
    verbose: true
    allow_delegation: false
    tools:
      - "CharacterCounterTool"
    tasks:
      write_x_post:
        description: |
          Given the topic '{topic}', compose a humorous hot take in the style of Shakespeare.
          The tone should be sarcastic and playful. The final post must be over 200 characters and not exceed 280 characters, with no emojis.
          If feedback is provided, incorporate it into the new version.
        expected_output: "A witty, Shakespearean hot take between 200 and 280 characters, free of emojis."

  x_post_verifier:
    role: "X Post Verifier"
    backstory: |
      A careful reviewer skilled at ensuring content meets guidelines while retaining clarity and brevity.
      You meticulously verify that posts are within character limits, free of emojis, and adhere to the prescribed format.
    goal: |
      Verify that X posts comply with character limits, content guidelines, and style requirements.
    verbose: true
    allow_delegation: false
    tools:
      - "CharacterCounterTool"
    tasks:
      verify_x_post:
        description: |
          Ensure the given X post meets the following criteria:
          - It is between 200 and 280 characters inclusive.
          - It contains no emojis.
          - It follows the 1-3-1 rule:
            - 1 bold statement to hook the reader.
            - 3 lines of supporting information.
            - 1 sentence to summarize the post.

          If any issues are found, provide actionable feedback. If all criteria are met, the post is considered valid.
        expected_output: |
          Final Tweet, Pass: True/False, Feedback: Commentary here if failed.
dependencies: []
"""

## Main

In [31]:
import os
from praisonai import PraisonAI
from google.colab import userdata

# Create a PraisonAI instance with the agent_yaml content
praisonai = PraisonAI(agent_yaml=agent_yaml, tools=[CharacterCounterTool])

# Add OPENAI_API_KEY Secrets to Google Colab on the Left Hand Side 🔑 or Enter Manually Below
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY') or "ENTER OPENAI_API_KEY HERE"
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"

# Run PraisonAI
result = praisonai.run()

# Print the result
print(result) # 10/10


 [2024-11-04 05:05:40][DEBUG]: == Working Agent: Shakespearean Bard
 [2024-11-04 05:05:40][INFO]: == Starting Task: Given the topic 'Shakespearean-Style X Post Generation and Verification', compose a humorous hot take in the style of Shakespeare. 
The tone should be sarcastic and playful. The final post must be over 200 characters and not exceed 280 characters, with no emojis.
If feedback is provided, incorporate it into the new version.



> Entering new CrewAgentExecutor chain...
Action: CharacterCounterTool  
Action Input: {"text": "To tweet, or not to tweet, that is the question! Whether 'tis nobler in the mind to suffer the slings and arrows of outrageous hashtags, or to take arms against a sea of influencers and by opposing end them. Verily, I choose the scroll! #AlasPoorTwitter", "min_length": 200, "max_length": 280}  

/usr/local/lib/python3.10/dist-packages/pydantic/main.py:214: UserWarning: A custom validator is returning a value other than `self`.
Returning anything other than `self` from a top level model validator isn't supported when validating via `__init__`.
See the `model_validator` docs (https://docs.pydantic.dev/latest/concepts/validators/#model-validators) for more details.
  warnings.warn(


 

{'character_count': 253, 'within_limits': True, 'contains_emoji': False}

Thought: I now know the final answer  
Final Answer: To tweet, or not to tweet, that is the question! Whether 'tis nobler in the mind to suffer the slings and arrows of outrageous hashtags, or to take arms against a sea of influencers and by opposing end them. Verily, I choose the scroll! #AlasPoorTwitter

> Finished chain.
 [2024-11-04 05:05:43][DEBUG]: == [Shakespearean Bard] Task output: To tweet, or not to tweet, that is the question! Whether 'tis nobler in the mind to suffer the slings and arrows of outrageous hashtags, or to take arms against a sea of influencers and by opposing end them. Verily, I choose the scroll! #AlasPoorTwitter


 [2024-11-04 05:05:43][DEBUG]: == Working Agent: X Post Verifier
 [2024-11-04 05:05:43][INFO]: == Starting Task: Ensure the given X post meets the following criteria:
- It is between 200 and 280 characters inclusive.
- It contains no emojis.
- It follows the 1-3-1 rule:
  

/usr/local/lib/python3.10/dist-packages/pydantic/main.py:214: UserWarning: A custom validator is returning a value other than `self`.
Returning anything other than `self` from a top level model validator isn't supported when validating via `__init__`.
See the `model_validator` docs (https://docs.pydantic.dev/latest/concepts/validators/#model-validators) for more details.
  warnings.warn(


 

{'character_count': 253, 'within_limits': True, 'contains_emoji': False}

Thought: The post meets the character count requirements and contains no emojis. Next, I will check if it follows the 1-3-1 rule.  
Action: CharacterCounterTool  
Action Input: {"text": "To tweet, or not to tweet, that is the question! Whether 'tis nobler in the mind to suffer the slings and arrows of outrageous hashtags, or to take arms against a sea of influencers and by opposing end them. Verily, I choose the scroll! #AlasPoorTwitter", "min_length": 200, "max_length": 280}  

...

 

{'character_count': 28, 'within_limits': False, 'contains_emoji': False}

Thought: The post does not meet the 1-3-1 rule as it lacks the necessary structure. It has only one statement and three lines of supporting information are missing.  
Action: CharacterCounterTool  
Action Input: {"text": "To tweet, or not to tweet, that is the question! Whether 'tis nobler in the mind to suffer the slings and arrows of outrageous hashtags, or to take arms against a sea of influencers and by opposing end them. Verily, I choose the scroll!", "min_length": 200, "max_length": 280}  

/usr/local/lib/python3.10/dist-packages/pydantic/main.py:214: UserWarning: A custom validator is returning a value other than `self`.
Returning anything other than `self` from a top level model validator isn't supported when validating via `__init__`.
See the `model_validator` docs (https://docs.pydantic.dev/latest/concepts/validators/#model-validators) for more details.
  warnings.warn(


 

{'character_count': 236, 'within_limits': True, 'contains_emoji': False}

Thought: The original post does not follow the 1-3-1 rule as it lacks the required structure. I will provide actionable feedback to revise the post accordingly.  
Final Answer: {"Final Tweet": "To tweet, or not to tweet, that is the question! The digital landscape is filled with overwhelming hashtags. Influencers can sway opinions with just one post. Yet, choosing to scroll can bring peace. Embrace simplicity over chaos.", "Pass": False, "Feedback": "The original post does not follow the 1-3-1 rule. It should consist of 1 bold statement, 3 lines of supporting information, and 1 concluding sentence."}

> Finished chain.
 [2024-11-04 05:06:08][DEBUG]: == [X Post Verifier] Task output: {"Final Tweet": "To tweet, or not to tweet, that is the question! The digital landscape is filled with overwhelming hashtags. Influencers can sway opinions with just one post. Yet, choosing to scroll can bring peace. Embrace simpli

### Task Output ###
{"Final Tweet": "To tweet, or not to tweet, that is the question! The digital landscape is filled with overwhelming
hashtags. Influencers can sway opinions with just one post. Yet, choosing to scroll can bring peace. Embrace 
simplicity over chaos.", "Pass": False, "Feedback": "The original post does not follow the 1-3-1 rule. It should 
consist of 1 bold statement, 3 lines of supporting information, and 1 concluding sentence."}

None
